In [48]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn-poster')
import warnings; warnings.simplefilter('ignore')

In [49]:
data = pd.read_csv('lyrics_titles_AutoPump.csv')

In [50]:
all_lyrics = open('all_lyrics.txt', 'w')

In [120]:
data.head()

,Unnamed: 0,title,lyrics
0,0,1 Choppa 2 Choppa*,"[Intro]\r\nAyy, ayy\r\n\r\n[Chorus]\r\nOne cho..."
1,0,30's,"[Verse 1: Lil Pump]\r\nI'ma hit a stain, I'ma ..."
2,0,70 Nigga,\r\n Lyrics for this song...
3,0,Aight,"[Intro: Lil Pump]\r\nYuh, ouu, ouu, ouu, ouu, ..."
4,0,At the Door,"[Intro]\r\nOoh, Big Head on the beat\r\nLil Pu..."


In [52]:
for row in data.itertuples():
    text = row.lyrics
    all_lyrics.write(text)

In [53]:
all_lyrics.close()

In [54]:
all_lyrics = open('all_lyrics.txt', 'r')
for line in all_lyrics:
    if line.startswith('  '):
        print(line)

            Lyrics for this song have yet to be released. Please check back once the song has been released.

          

        [Intro: Lil Pump]

            Lyrics for this song have yet to be released. Please check back once the song has been released.

          

        [Intro: Lil Pump]

            Lyrics for this song have yet to be released. Please check back once the song has been released.

          

        [Verse]          

            Lyrics for this song have yet to be released. Please check back once the song has been released.

          

        [Intro]

            Lyrics for this song have yet to be released. Please check back once the song has been released.

          

        [Intro: Rick Ross]

            Lyrics for this song have yet to be released. Please check back once the song has been released.

          

        [Intro]

            Lyrics for this song have yet to be released. Please check back once the song has been released.

          

   

In [101]:
lines = []
all_lyrics = open('all_lyrics.txt', 'r')
for line in all_lyrics:
    if line.startswith(' '):
        line = line.lstrip()
    line = line.lower()
    if line.startswith('[chorus'):
        lines.append('[chorus] \n')
    elif line.startswith('[verse'):
        lines.append('[verse] \n')
    elif line.startswith('[hook'):
        lines.append('[hook] \n')
    elif line.startswith('[intro'):
        lines.append('[intro] \n')
    elif line.startswith('[outro'):
        lines.append('[outro] \n')
    elif line.startswith('[bridge'):
        lines.append('[bridge] \n')
    elif line.startswith('[interlude'):
        lines.append('[interlude] \n')
    elif line.startswith('lyrics for this'):
        lines.append('\n')
    else:
        lines.append(line)
all_lyrics.close()

In [102]:
all_lyrics_cleaned = open('all_lyrics_cleaned.txt', 'w')
for line in lines:
    all_lyrics_cleaned.write(line)
all_lyrics_cleaned.close() 

In [115]:
# Read in the entire file.
f = open('all_lyrics_cleaned.txt')
corpus0 = f.read()
f.close()

# Separate the punctuation from the words, so that words with
# punctuation do not get counted as distinct from words without
# punctuation.  Same for new line characters.

corpus0 = corpus0.replace(',', ' ,')
corpus0 = corpus0.replace('(', ' ( ')
corpus0 = corpus0.replace(')', ' ) ')
corpus0 = corpus0.replace('[', ' [ ')
corpus0 = corpus0.replace(']', ' ] ')
corpus0 = corpus0.replace('.', ' . ')
corpus0 = corpus0.replace(';', ' ; ')
corpus0 = corpus0.replace(':', ' : ')
corpus0 = corpus0.replace('!', ' ! ')
corpus0 = corpus0.replace('?', ' ? ')
corpus0 = corpus0.replace('*', ' * ')
corpus0 = corpus0.replace("’", '\'')
corpus0 = corpus0.replace("\'\'", ' " ')
corpus0 = corpus0.replace('"', ' " ')
corpus0 = corpus0.replace('\r\n', ' \r\n ')

# Separate the dashes from any words they're attached to.
corpus0 = corpus0.replace('-', ' - ')
corpus0 = corpus0.replace('\n', ' \n ')

# Convert the text to lower case.
corpus0 = corpus0.lower()

# Split the words by spaces; only take the first 500000 words.
# This number was chosen based on memory limits and training-time
# limitations.
corpus1 = corpus0.split(' ')

while (corpus1.count('') > 0): 
    corpus1.remove('')
    
print('Length of corpus is now ', len(corpus1))

Length of corpus is now  46686


In [116]:
# Preprocessing is done.  Now get the unique words, and encode them.
words = sorted(list(set(corpus1)))
num_words = len(words)
encoding = {w: i for i, w in enumerate(words)}
decoding = {i: w for i, w in enumerate(words)}

print('We have', num_words, 'different words.')
corpus = corpus1

We have 2476 different words.


In [123]:
# Chop up the data into x and y, slice into roughly num_chars
# overlapping 'sentences' of length sentence_length.  Encode the
# characters.
sentence_length = 20
x_data = []
y_data = []
for i in range(0, len(corpus) - sentence_length):
    sentence = corpus[i: i + sentence_length]
    next_word = corpus[i + sentence_length]
    x_data.append([encoding[word] for word in sentence])
    y_data.append(encoding[next_word])

# good word: phronesis
num_sentences = len(x_data)
print('We have', len(x_data), 'sentences.')

# Create the variables to hold the data as it will be used.
x = np.zeros((num_sentences, sentence_length, num_words), dtype = np.bool)
y = np.zeros((num_sentences, num_words), dtype = np.bool)

# Populate the sentences.
print('Encoding data.')
for i, sentence in enumerate(x_data):
    for t, encoded_word in enumerate(sentence):
        x[i, t, encoded_word] = 1
    y[i, y_data[i]] = 1

We have 46666 sentences.
Encoding data.


In [124]:
# The processing of the data takes a fair amount of time.  Save
# the data so we don't have to do this again.  We do this in a
# numpy file since the data is large and the shelve can't handle
# it.

print('Saving processed data.')
np.save('x.npy', x)
np.save('y.npy', y)

Saving processed data.


In [125]:
import keras.models as km
import keras.layers as kl

Using TensorFlow backend.


In [126]:
print('Building network.')
model = km.Sequential()
model.add(kl.LSTM(100, input_shape = (sentence_length, num_words)))
model.add(kl.Dense(num_words, activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'sgd',
              metrics = ['accuracy'])

Building network.


In [128]:
# Fit!  Begin elevator music...
print('Beginning fit.')
fit = model.fit(x, y, epochs = 200, batch_size = 128)

# Save the model so that we can use it as a starting point.
model.save('model.h5')

Beginning fit.
Epoch 1/200
19712/46666 [===========>..................] - ETA: 1:37 - loss: 7.7872 - acc: 0.0980

KeyboardInterrupt: 